In [1]:
import numpy as np
import pandas as pd
import time
import math
import datetime
import os
import shutil
import cv2
import skimage
import skimage.transform
from skimage.transform import rescale
from scipy.signal import convolve2d
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg


Option 1:
0. Normal
1. Flip horizontal

Option 2:
0. Normal
1. Contrast +15
2. Contrast -15
3. Brightness +20
4. Brightness -20

Option 3:
0. Normal
1. Rotation +15deg
2. Rotation -15deg

Option 4:
0. Normal
1. Gaussian Blur
2. Gaussian Noise

In [2]:
# opt1 = ['normal', 'flip']
opt2 = ['normal', 'contrast+', 'contrast-', 'brightness+', 'brightness-']
opt3 = ['normal', 'rotate+', 'rotate-']
opt4 = ['normal', 'blur', 'noise', 'sharpen']

In [3]:
import random

#max combination = 2 * 5 * 3 * 3 = 150 - 1 = 149
def generate_random_combination(n_combination=10):
    combination = []
    
    itr = 0
    
    while itr < n_combination:
#         temp = [random.randrange(0, len(opt1)), 
        temp = [random.randrange(0, len(opt2)),
               random.randrange(0, len(opt3)), random.randrange(0, len(opt4))]
        
        if temp == [0, 0, 0]:
            continue
            
        if itr == 0:
            combination.append(temp)
            itr +=1
        else:
            isSame = False
            for data in combination:
                if data == temp:
                    isSame = True
                    break
            if not isSame:
                combination.append(temp)
                itr += 1
            else:
                continue
    
    return np.asarray(combination)

In [4]:
path_data = 'D:/belajar/PA/dataset/[3-splitted]/'
dirs = os.listdir(path_data)
print(dirs)

['20220201', '20220228', '20220326', '20220327']


In [5]:
# ISI INI YA!
DATE_PICK = '20220327'
sub_dir = DATE_PICK + '/train/'
path_data = path_data + sub_dir

In [6]:
labels = range(0, 7)
today_date = datetime.datetime.now().strftime("%Y%m%d")
base_dst_folder = 'D:/belajar/PA/dataset/[4-augmented]/'+today_date+'/'

In [7]:
os.mkdir(base_dst_folder)

In [8]:
for label in labels:
    os.mkdir(base_dst_folder+'/'+str(label))

In [9]:
raw_data = pd.DataFrame(columns=["label", "path"])

for label in labels:
    folder = path_data + str(label) + '/'
    files = os.listdir(folder)
    for file in files:
        raw_data.loc[len(raw_data.index)] = [label, file]

In [10]:
def my_round(num, precise=0.2):
    if num-precise >= np.floor(num):
        return np.ceil(num)
    return np.floor(num)

In [11]:
def aug_calc(data_len, target):
    return my_round(target/data_len)

In [12]:
def flip(img):
    return cv2.flip(img, 1) #horizontally

def contrast(img, alp=0.1):
    alp += 1
    return cv2.convertScaleAbs(img, alpha=alp)

def brightness(img, beta=10):
    return cv2.convertScaleAbs(img, beta=beta)

def rotate(img, deg=10):
    return skimage.transform.rotate(img, deg, resize=False)

def gaussian_blur(img):
    return cv2.GaussianBlur(img,(3, 3),0)

def gaussian_noise(img, var=0.002):
    return skimage.util.random_noise(img, mode='gaussian', var=var)

def sharpen(img):
    kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
    return cv2.filter2D(src=img, ddepth=-1, kernel=kernel)
#     return cv2.filter2D(src=img, ddepth=-1, kernel=kernel)


In [13]:
def rgb_convolve2d(image, kernel):
    red = convolve2d(image[:,:,0], kernel, 'valid')
    green = convolve2d(image[:,:,1], kernel, 'valid')
    blue = convolve2d(image[:,:,2], kernel, 'valid')
    return np.stack([red, green, blue], axis=2)

In [14]:
scale_pr = 1

In [15]:
def sharpen10(image):
    r_scaled = rescale(image[:,:,0], scale_pr)
    g_scaled = rescale(image[:,:,1], scale_pr)
    b_scaled = rescale(image[:,:,2], scale_pr)
    my_dog_scaled = np.stack([r_scaled, g_scaled, b_scaled], axis=2)

    kernel = np.array([[0, -1, 0],
                        [-1, 5, -1],
                        [0, -1, 0]])
    return rgb_convolve2d(my_dog_scaled, kernel[::-1, ::-1]).clip(0,1)

In [16]:
def show(image):
    plt.imshow(image)
    plt.show()
    time.sleep(1)
    plt.close()



In [17]:
def normalize(image):
    normalized = image.astype(float)
    min = normalized.min()
    max = normalized.max()
    normalized = np.round(255 * (normalized - min)/(max - min))
    normalized = normalized.astype(np.uint8)

    return normalized

In [18]:
def write_np(data, name):
    # Write the array to disk
    with open(name+'.txt', 'w') as outfile:
        # I'm writing a header here just for the sake of readability
        # Any line starting with "#" will be ignored by numpy.loadtxt
        outfile.write('# Array shape: {0}\n'.format(data.shape))

        # Iterating through a ndimensional array produces slices along
        # the last axis. This is equivalent to data[i,:,:] in this case
        for data_slice in data:

            # The formatting string indicates that I'm writing out
            # the values in left-justified columns 7 characters in width
            # with 2 decimal places.  
            np.savetxt(outfile, data_slice, fmt='%-7.2f')

            # Writing out a break to indicate different slices...
            outfile.write('# New slice\n')
    print("done" + name)

In [19]:
PIXEL_PICTURE = 128
NUM_TARGET = PIXEL_PICTURE * PIXEL_PICTURE
NUM_AUG_PER_CLASS = math.ceil(NUM_TARGET / len(labels))
n_combs = []
combinations = []
for label in labels:
    x = raw_data.loc[raw_data['label'] == label].count().path
    n = aug_calc(x, NUM_AUG_PER_CLASS)
    n_combs.append(n)
    print(f"class {label}: {x} data, expected {n} augment")
    combinations.append(generate_random_combination(n))

print(n_combs)
print(combinations)

class 0: 345 data, expected 7.0 augment
class 1: 364 data, expected 7.0 augment
class 2: 369 data, expected 7.0 augment
class 3: 318 data, expected 8.0 augment
class 4: 326 data, expected 7.0 augment
class 5: 237 data, expected 10.0 augment
class 6: 333 data, expected 7.0 augment
[7.0, 7.0, 7.0, 8.0, 7.0, 10.0, 7.0]
[array([[1, 1, 1],
       [3, 1, 2],
       [3, 2, 1],
       [4, 2, 2],
       [1, 0, 3],
       [4, 2, 3],
       [2, 1, 2]]), array([[4, 0, 3],
       [0, 1, 2],
       [4, 1, 2],
       [1, 0, 2],
       [2, 1, 3],
       [1, 1, 1],
       [1, 0, 0]]), array([[4, 0, 3],
       [2, 2, 1],
       [2, 2, 2],
       [4, 2, 3],
       [0, 2, 3],
       [2, 1, 3],
       [1, 0, 1]]), array([[1, 2, 2],
       [4, 1, 0],
       [1, 1, 2],
       [2, 2, 0],
       [0, 2, 2],
       [1, 1, 1],
       [2, 1, 1],
       [1, 1, 3]]), array([[1, 0, 1],
       [2, 1, 3],
       [0, 2, 0],
       [2, 0, 1],
       [2, 0, 3],
       [2, 0, 2],
       [4, 2, 2]]), array([[4, 1, 2],
     

In [20]:
def augment(data, combinations=[]):
    i = 0
    for id, row in data.iterrows():
        src = path_data + str(row.label) + '/' + row.path
#         print(src)
#         print(id)
        image = cv2.imread(src)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = normalize(image)
        file_name = os.path.splitext(os.path.basename(row.path))[0]
        print(file_name)
        for combination in combinations:
            filter_img = image
            
            two_comb = combination[0]
            print(opt2[two_comb])
            if two_comb == 1:
                filter_img = contrast(filter_img, 0.15)
            elif two_comb == 2:
                filter_img = contrast(filter_img, -0.10)
            elif two_comb == 3:
                filter_img = brightness(filter_img, 20)
            elif two_comb == 4:
                filter_img = brightness(filter_img, -10)
#             print(filter_img)
#             print("=========")            
            
            three_comb = combination[1]
            print(opt3[three_comb])
            if three_comb == 1:
                filter_img = rotate(filter_img, 15)
            elif three_comb == 2:
                filter_img = rotate(filter_img, -15)
#             show(filter_img)
#             print(filter_img)
#             print("=========")
    
            forth_comb = combination[2]
            print(opt4[forth_comb])
            if forth_comb == 1:
                filter_img = gaussian_blur(filter_img)
            elif forth_comb == 2:
                filter_img = gaussian_noise(filter_img)
            elif forth_comb == 3:
                filter_img = sharpen10(filter_img)
#             show(filter_img)
#             filter_img = normalize(filter_img)
#             print(filter_img)
#             show(filter_img)
            print("=========")
            
#             print(filter_img)
            print(file_name)

#             print(filter_img)
#             filter_img = np.round(cv2.resize(filter_img, (128, 128), interpolation = cv2.INTER_AREA))
#             filter_img = filter_img.astype(np.uint8)
            dst = base_dst_folder + str(row.label) + '/' + file_name + str(i) + '.jpg'
            plt.imsave(dst, filter_img)
            i+=1

In [21]:
for label in labels:
    tmp_data = raw_data.loc[raw_data['label'] == label]
    print(combinations[label])
    augment(tmp_data, combinations[label])
    print(f'=====saved {label} successfully=====')

[[1 1 1]
 [3 1 2]
 [3 2 1]
 [4 2 2]
 [1 0 3]
 [4 2 3]
 [2 1 2]]
AF01ANHL
contrast+
rotate+
blur
AF01ANHL
brightness+
rotate+
noise
AF01ANHL
brightness+
rotate-
blur
AF01ANHL
brightness-
rotate-
noise
AF01ANHL
contrast+
normal
sharpen
AF01ANHL
brightness-
rotate-
sharpen
AF01ANHL
contrast-
rotate+
noise
AF01ANHL
AF01ANHR
contrast+
rotate+
blur
AF01ANHR
brightness+
rotate+
noise
AF01ANHR
brightness+
rotate-
blur
AF01ANHR
brightness-
rotate-
noise
AF01ANHR
contrast+
normal
sharpen
AF01ANHR
brightness-
rotate-
sharpen
AF01ANHR
contrast-
rotate+
noise
AF01ANHR
AF01ANS
contrast+
rotate+
blur
AF01ANS
brightness+
rotate+
noise
AF01ANS
brightness+
rotate-
blur
AF01ANS
brightness-
rotate-
noise
AF01ANS
contrast+
normal
sharpen
AF01ANS
brightness-
rotate-
sharpen
AF01ANS
contrast-
rotate+
noise
AF01ANS
AF02ANHL
contrast+
rotate+
blur
AF02ANHL
brightness+
rotate+
noise
AF02ANHL
brightness+
rotate-
blur
AF02ANHL
brightness-
rotate-
noise
AF02ANHL
contrast+
normal
sharpen
AF02ANHL
brightness-
rotate

rotate-
sharpen
AF10ANS
contrast-
rotate+
noise
AF10ANS
AF11ANHL
contrast+
rotate+
blur
AF11ANHL
brightness+
rotate+
noise
AF11ANHL
brightness+
rotate-
blur
AF11ANHL
brightness-
rotate-
noise
AF11ANHL
contrast+
normal
sharpen
AF11ANHL
brightness-
rotate-
sharpen
AF11ANHL
contrast-
rotate+
noise
AF11ANHL
AF11ANHR
contrast+
rotate+
blur
AF11ANHR
brightness+
rotate+
noise
AF11ANHR
brightness+
rotate-
blur
AF11ANHR
brightness-
rotate-
noise
AF11ANHR
contrast+
normal
sharpen
AF11ANHR
brightness-
rotate-
sharpen
AF11ANHR
contrast-
rotate+
noise
AF11ANHR
AF11ANS
contrast+
rotate+
blur
AF11ANS
brightness+
rotate+
noise
AF11ANS
brightness+
rotate-
blur
AF11ANS
brightness-
rotate-
noise
AF11ANS
contrast+
normal
sharpen
AF11ANS
brightness-
rotate-
sharpen
AF11ANS
contrast-
rotate+
noise
AF11ANS
AF12ANHR
contrast+
rotate+
blur
AF12ANHR
brightness+
rotate+
noise
AF12ANHR
brightness+
rotate-
blur
AF12ANHR
brightness-
rotate-
noise
AF12ANHR
contrast+
normal
sharpen
AF12ANHR
brightness-
rotate-
sharpe

AF20ANS
contrast-
rotate+
noise
AF20ANS
AF21ANHL
contrast+
rotate+
blur
AF21ANHL
brightness+
rotate+
noise
AF21ANHL
brightness+
rotate-
blur
AF21ANHL
brightness-
rotate-
noise
AF21ANHL
contrast+
normal
sharpen
AF21ANHL
brightness-
rotate-
sharpen
AF21ANHL
contrast-
rotate+
noise
AF21ANHL
AF21ANHR
contrast+
rotate+
blur
AF21ANHR
brightness+
rotate+
noise
AF21ANHR
brightness+
rotate-
blur
AF21ANHR
brightness-
rotate-
noise
AF21ANHR
contrast+
normal
sharpen
AF21ANHR
brightness-
rotate-
sharpen
AF21ANHR
contrast-
rotate+
noise
AF21ANHR
AF21ANS
contrast+
rotate+
blur
AF21ANS
brightness+
rotate+
noise
AF21ANS
brightness+
rotate-
blur
AF21ANS
brightness-
rotate-
noise
AF21ANS
contrast+
normal
sharpen
AF21ANS
brightness-
rotate-
sharpen
AF21ANS
contrast-
rotate+
noise
AF21ANS
AF23ANHL
contrast+
rotate+
blur
AF23ANHL
brightness+
rotate+
noise
AF23ANHL
brightness+
rotate-
blur
AF23ANHL
brightness-
rotate-
noise
AF23ANHL
contrast+
normal
sharpen
AF23ANHL
brightness-
rotate-
sharpen
AF23ANHL
contr

AF31ANHR
contrast-
rotate+
noise
AF31ANHR
AF31ANS
contrast+
rotate+
blur
AF31ANS
brightness+
rotate+
noise
AF31ANS
brightness+
rotate-
blur
AF31ANS
brightness-
rotate-
noise
AF31ANS
contrast+
normal
sharpen
AF31ANS
brightness-
rotate-
sharpen
AF31ANS
contrast-
rotate+
noise
AF31ANS
AF32ANS
contrast+
rotate+
blur
AF32ANS
brightness+
rotate+
noise
AF32ANS
brightness+
rotate-
blur
AF32ANS
brightness-
rotate-
noise
AF32ANS
contrast+
normal
sharpen
AF32ANS
brightness-
rotate-
sharpen
AF32ANS
contrast-
rotate+
noise
AF32ANS
AF33ANHL
contrast+
rotate+
blur
AF33ANHL
brightness+
rotate+
noise
AF33ANHL
brightness+
rotate-
blur
AF33ANHL
brightness-
rotate-
noise
AF33ANHL
contrast+
normal
sharpen
AF33ANHL
brightness-
rotate-
sharpen
AF33ANHL
contrast-
rotate+
noise
AF33ANHL
AF33ANHR
contrast+
rotate+
blur
AF33ANHR
brightness+
rotate+
noise
AF33ANHR
brightness+
rotate-
blur
AF33ANHR
brightness-
rotate-
noise
AF33ANHR
contrast+
normal
sharpen
AF33ANHR
brightness-
rotate-
sharpen
AF33ANHR
contrast-
r

AM08ANHR
contrast-
rotate+
noise
AM08ANHR
AM08ANS
contrast+
rotate+
blur
AM08ANS
brightness+
rotate+
noise
AM08ANS
brightness+
rotate-
blur
AM08ANS
brightness-
rotate-
noise
AM08ANS
contrast+
normal
sharpen
AM08ANS
brightness-
rotate-
sharpen
AM08ANS
contrast-
rotate+
noise
AM08ANS
AM09ANHL
contrast+
rotate+
blur
AM09ANHL
brightness+
rotate+
noise
AM09ANHL
brightness+
rotate-
blur
AM09ANHL
brightness-
rotate-
noise
AM09ANHL
contrast+
normal
sharpen
AM09ANHL
brightness-
rotate-
sharpen
AM09ANHL
contrast-
rotate+
noise
AM09ANHL
AM09ANHR
contrast+
rotate+
blur
AM09ANHR
brightness+
rotate+
noise
AM09ANHR
brightness+
rotate-
blur
AM09ANHR
brightness-
rotate-
noise
AM09ANHR
contrast+
normal
sharpen
AM09ANHR
brightness-
rotate-
sharpen
AM09ANHR
contrast-
rotate+
noise
AM09ANHR
AM09ANS
contrast+
rotate+
blur
AM09ANS
brightness+
rotate+
noise
AM09ANS
brightness+
rotate-
blur
AM09ANS
brightness-
rotate-
noise
AM09ANS
contrast+
normal
sharpen
AM09ANS
brightness-
rotate-
sharpen
AM09ANS
contrast-


...

sharpen
BF22NEHR
brightness-
rotate+
sharpen
BF22NEHR
brightness+
rotate+
normal
BF22NEHR
brightness-
rotate-
noise
BF22NEHR
brightness-
normal
normal
BF22NEHR
contrast+
rotate-
noise
BF22NEHR
contrast+
rotate-
sharpen
BF22NEHR
BF22NES
brightness-
rotate-
sharpen
BF22NES
brightness-
rotate+
sharpen
BF22NES
brightness+
rotate+
normal
BF22NES
brightness-
rotate-
noise
BF22NES
brightness-
normal
normal
BF22NES
contrast+
rotate-
noise
BF22NES
contrast+
rotate-
sharpen
BF22NES
BF23NEHL
brightness-
rotate-
sharpen
BF23NEHL
brightness-
rotate+
sharpen
BF23NEHL
brightness+
rotate+
normal
BF23NEHL
brightness-
rotate-
noise
BF23NEHL
brightness-
normal
normal
BF23NEHL
contrast+
rotate-
noise
BF23NEHL
contrast+
rotate-
sharpen
BF23NEHL
BF23NEHR
brightness-
rotate-
sharpen
BF23NEHR
brightness-
rotate+
sharpen
BF23NEHR
brightness+
rotate+
normal
BF23NEHR
brightness-
rotate-
noise
BF23NEHR
brightness-
normal
normal
BF23NEHR
contrast+
rotate-
noise
BF23NEHR
contrast+
rotate-
sharpen
BF23NEHR
BF23NES
b

noise
BF31NES
brightness-
normal
normal
BF31NES
contrast+
rotate-
noise
BF31NES
contrast+
rotate-
sharpen
BF31NES
BF32NEHL
brightness-
rotate-
sharpen
BF32NEHL
brightness-
rotate+
sharpen
BF32NEHL
brightness+
rotate+
normal
BF32NEHL
brightness-
rotate-
noise
BF32NEHL
brightness-
normal
normal
BF32NEHL
contrast+
rotate-
noise
BF32NEHL
contrast+
rotate-
sharpen
BF32NEHL
BF32NEHR
brightness-
rotate-
sharpen
BF32NEHR
brightness-
rotate+
sharpen
BF32NEHR
brightness+
rotate+
normal
BF32NEHR
brightness-
rotate-
noise
BF32NEHR
brightness-
normal
normal
BF32NEHR
contrast+
rotate-
noise
BF32NEHR
contrast+
rotate-
sharpen
BF32NEHR
BF32NES
brightness-
rotate-
sharpen
BF32NES
brightness-
rotate+
sharpen
BF32NES
brightness+
rotate+
normal
BF32NES
brightness-
rotate-
noise
BF32NES
brightness-
normal
normal
BF32NES
contrast+
rotate-
noise
BF32NES
contrast+
rotate-
sharpen
BF32NES
BF33NEHL
brightness-
rotate-
sharpen
BF33NEHL
brightness-
rotate+
sharpen
BF33NEHL
brightness+
rotate+
normal
BF33NEHL
brig

contrast+
rotate-
sharpen
BM06NES
BM07NEHR
brightness-
rotate-
sharpen
BM07NEHR
brightness-
rotate+
sharpen
BM07NEHR
brightness+
rotate+
normal
BM07NEHR
brightness-
rotate-
noise
BM07NEHR
brightness-
normal
normal
BM07NEHR
contrast+
rotate-
noise
BM07NEHR
contrast+
rotate-
sharpen
BM07NEHR
BM07NES
brightness-
rotate-
sharpen
BM07NES
brightness-
rotate+
sharpen
BM07NES
brightness+
rotate+
normal
BM07NES
brightness-
rotate-
noise
BM07NES
brightness-
normal
normal
BM07NES
contrast+
rotate-
noise
BM07NES
contrast+
rotate-
sharpen
BM07NES
BM08NEHL
brightness-
rotate-
sharpen
BM08NEHL
brightness-
rotate+
sharpen
BM08NEHL
brightness+
rotate+
normal
BM08NEHL
brightness-
rotate-
noise
BM08NEHL
brightness-
normal
normal
BM08NEHL
contrast+
rotate-
noise
BM08NEHL
contrast+
rotate-
sharpen
BM08NEHL
BM08NES
brightness-
rotate-
sharpen
BM08NES
brightness-
rotate+
sharpen
BM08NES
brightness+
rotate+
normal
BM08NES
brightness-
rotate-
noise
BM08NES
brightness-
normal
normal
BM08NES
contrast+
rotate-
no

BM17NEHR
brightness+
rotate+
normal
BM17NEHR
brightness-
rotate-
noise
BM17NEHR
brightness-
normal
normal
BM17NEHR
contrast+
rotate-
noise
BM17NEHR
contrast+
rotate-
sharpen
BM17NEHR
BM18NEHR
brightness-
rotate-
sharpen
BM18NEHR
brightness-
rotate+
sharpen
BM18NEHR
brightness+
rotate+
normal
BM18NEHR
brightness-
rotate-
noise
BM18NEHR
brightness-
normal
normal
BM18NEHR
contrast+
rotate-
noise
BM18NEHR
contrast+
rotate-
sharpen
BM18NEHR
BM18NES
brightness-
rotate-
sharpen
BM18NES
brightness-
rotate+
sharpen
BM18NES
brightness+
rotate+
normal
BM18NES
brightness-
rotate-
noise
BM18NES
brightness-
normal
normal
BM18NES
contrast+
rotate-
noise
BM18NES
contrast+
rotate-
sharpen
BM18NES
BM19NEHL
brightness-
rotate-
sharpen
BM19NEHL
brightness-
rotate+
sharpen
BM19NEHL
brightness+
rotate+
normal
BM19NEHL
brightness-
rotate-
noise
BM19NEHL
brightness-
normal
normal
BM19NEHL
contrast+
rotate-
noise
BM19NEHL
contrast+
rotate-
sharpen
BM19NEHL
BM19NES
brightness-
rotate-
sharpen
BM19NES
brightness

BM29NEHL
contrast+
rotate-
sharpen
BM29NEHL
BM29NEHR
brightness-
rotate-
sharpen
BM29NEHR
brightness-
rotate+
sharpen
BM29NEHR
brightness+
rotate+
normal
BM29NEHR
brightness-
rotate-
noise
BM29NEHR
brightness-
normal
normal
BM29NEHR
contrast+
rotate-
noise
BM29NEHR
contrast+
rotate-
sharpen
BM29NEHR
BM29NES
brightness-
rotate-
sharpen
BM29NES
brightness-
rotate+
sharpen
BM29NES
brightness+
rotate+
normal
BM29NES
brightness-
rotate-
noise
BM29NES
brightness-
normal
normal
BM29NES
contrast+
rotate-
noise
BM29NES
contrast+
rotate-
sharpen
BM29NES
BM30NEHR
brightness-
rotate-
sharpen
BM30NEHR
brightness-
rotate+
sharpen
BM30NEHR
brightness+
rotate+
normal
BM30NEHR
brightness-
rotate-
noise
BM30NEHR
brightness-
normal
normal
BM30NEHR
contrast+
rotate-
noise
BM30NEHR
contrast+
rotate-
sharpen
BM30NEHR
BM30NES
brightness-
rotate-
sharpen
BM30NES
brightness-
rotate+
sharpen
BM30NES
brightness+
rotate+
normal
BM30NES
brightness-
rotate-
noise
BM30NES
brightness-
normal
normal
BM30NES
contrast+


In [22]:
print("DONE!")

DONE!


In [23]:
# # import cv2
# # import math

# src = 'D:/belajar/PA/dataset/[3-splitted]/train/0/AF01ANHL.jpg'
# img = cv2.imread(src)

# # Convert into grayscale
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# # img = cv2.resize(img, (100, 100), interpolation = cv2.INTER_AREA)
# # show(img)
# image = normalize(img)
# print("@@@@@@@@@@@@@@@@@@@@@@@")
# # print(image)
# show(image)

# rt_img = rotate(image, -15)
# rt_norm = rt_img * (np.max(rt_img) - np.min(rt_img)) + np.min(rt_img)
# rt_norm = rt_norm.astype(int)
# print("@@@@@@@@@@@@@@@@@@@@@@@")
# print("rotate")
# # print(rt_img)
# # show(rt_img)

# print("@@@@@@@@@@@@@@@@@@@@@@@")
# print("sharpen")
# shp_img = sharpen10(rt_img)
# print(shp_img)
# show(shp_img)

# # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

# dst = "D:/belajar/PA/dataset/[4-augmented]/1/lo.jpg"
# plt.imsave(dst, shp_img)
# # cv2.imwrite(dst, image)



# UJI